# House prices

## Todo

* Feature scaling
* Graphing
    * Try libraries like `seaborn`, `matplotlib`
* Better analysis methods
* (?) Use built-in pipeline CV rather than splitting manually
    * Hope this cuts down code complexity (slightly)
    * What graphs does this allow me to make?
* Backup this kernel on Github
    * Use the [Kaggle API](https://github.com/Kaggle/kaggle-api) tool to edit locally & run against remote images

In [ ]:
# Imports

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder
from xgboost.sklearn import XGBRegressor

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Load data

random_state=3
initial_data = pd.read_csv("../input/train.csv")
test_data = pd.read_csv("../input/test.csv")

# Align (unencoded) categoricals 
category_names = ['MSSubClass', 'MSZoning', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'MiscFeature']
for category_name in category_names:
    all_data = pd.concat([initial_data[category_name], test_data[category_name]])
    categories = pd.Series(all_data, dtype="category")
    initial_data[category_name] = initial_data[category_name].astype('category', categories)
    test_data[category_name] = test_data[category_name].astype('category', categories)

# Unnest DataFrames
initial_data[category_names] = initial_data[category_names].values
test_data[category_names] = test_data[category_names].values

# Set up predictors (/features)
predictor_names = initial_data.columns.drop(["Id", "SalePrice", "SaleType", "SaleCondition"]).values
X_initial = initial_data[predictor_names]
X_test = test_data[predictor_names]

# Set up target
target_name = "SalePrice"
y_initial = np.log(initial_data[target_name])

# Split datasets
X_train, X_validate, y_train, y_validate = train_test_split(X_initial, y_initial, random_state=random_state)

In [ ]:
# Examine data/features

print("Features")
print(predictor_names)
print("\n")

print(X_train.head())
print(X_train.describe())
print("\n")

print(y_train.head())
print(y_train.describe())
print("\n")

In [ ]:
# Build pipeline components

imputer = SimpleImputer()
encoder = OneHotEncoder()
createEstimator = (lambda params = {}: XGBRegressor(random_state=random_state, **params))

In [ ]:
# # Find optimal parameters (only needs running once for each model change)

# fixedParams = {
#     "n_estimators": 550,
#     "max_depth": 3,
# #     "early_stopping_rounds": 1,
# #     "learning_rate": 0.1,
#     "eval_set": [(X_validate, y_validate)],
# }

# base_pipeline = Pipeline([("encoder", encoder), ("imputer", imputer), ("estimator", createEstimator(fixedParams))])

# param_grid = {
# #     "estimator__n_estimators": [300, 500, 550, 600, 800],
#     "estimator__early_stopping_rounds": [1, 2, 3],
#     "estimator__learning_rate": [0.08, 0.1, 0.12],
# #     "estimator__max_depth": [1, 3, 5, 10],
# }
# grid = GridSearchCV(base_pipeline, param_grid, n_jobs=-1, cv=5)
# grid.fit(X_train, y_train)

# pipeline = grid.best_estimator_

# print(pipeline.get_params().keys())
# print(grid.best_score_)
# print(grid.best_params_)

In [ ]:
# Quick model training (only run once optimal parameters have been found)

params = {
    "n_estimators": 550,
    "max_depth": 3,
    "early_stopping_rounds": 1,
    "learning_rate": 0.1,
}
pipeline = Pipeline([("encoder", encoder), ("imputer", imputer), ("estimator", createEstimator(params))])
pipeline.fit(X_train, y_train)

In [ ]:
# Analyse

predictions = pipeline.predict(X_validate)

readable_predictions = np.exp(predictions)
readable_y_validate = np.exp(y_validate)

from sklearn.metrics import mean_squared_log_error
rmsle = np.sqrt(mean_squared_log_error(readable_y_validate, readable_predictions))
print("Root Mean Squared Log Error")
print(rmsle)
print("")

print("First 5")
print([float(x) for x in readable_predictions[:5]])
print([float(x) for x in readable_y_validate[:5]])
print("")

print("Last 5")
print([float(x) for x in readable_predictions[-5:]])
print([float(x) for x in readable_y_validate[-5:]])
print("")

In [ ]:
# Merge train & validation data for final model

X = pd.concat([X_train, X_validate])
y = pd.concat([y_train, y_validate])
pipeline.fit(X, y)

In [ ]:
# Submit
test_predictions = pipeline.predict(X_test)
submission = pd.DataFrame({'Id': test_data.Id, 'SalePrice': np.exp(test_predictions)})
submission.to_csv('submission.csv', index=False)

## Notes

* Don't need to manually split CV data with `StratifiedKFold` or similar, it's done automatically by `GridSearchCV` (defaults to 3-fold).

## References

* [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
* [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
* [Mean squared log error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_log_error.html)